# Import the necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nltk
#from nltk.stem.lancaster import LancasterStemmer
#stemmer = LancasterStemmer()
#nltk.download('wordnet')  WordNet is the lexical database i.e. dictionary for the English language, specifically 
#designed for natural language processing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
import csv

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








# Load the dataset

In [4]:
with open("diseases.json") as file:
    data = json.load(file)

# Data pre-processing

In [5]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [lemmatizer.lemmatize(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lemmatizer.lemmatize(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [6]:
symptoms=["hi","hello","bye","thanks","whats up","Hey"]
with open("diseases1.csv") as file1:
    symp = csv.reader(file1, delimiter=',')
    for row in symp:
        for s in row:
            symptoms.append(s)

# Create a DNN model

In [7]:
tensorflow.reset_default_graph() #Clears the default graph stack and resets the global default graph.

#We are building a 3-layers neural network using TFLearn. We need to specify the shape of our input data. In our case,
#each sample has a total of 318 features and we will process samples per batch to save memory, so our data 
#input shape is [None, 318] ('None' stands for an unknown dimension, so we can change the total number of samples 
#that are processed in a batch).
net = tflearn.input_data(shape=[None, len(training[0])]) #to know length of features we use len(training[0])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #Softmax function is used in the output layer of a 
                                    #neural net to represent a probability distribution of possible outcomes of the network.
net = tflearn.regression(net) #used in TFLearn to apply a regression (linear or logistic) to the provided input. It requires
                              #to specify a TensorFlow gradient descent optimizer 'optimizer' that will minimize the provided 
                              #loss function 'loss' '''

model = tflearn.DNN(net) #TFLearn provides a model wrapper 'DNN' that can automatically performs a neural network
                         #classifier tasks, such as training, prediction, save/restore, etc...'''


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# Fit and save the model

In [8]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True) #We will run it for 1000 epochs (the network 
                                                                #will see all data 1000 times) with a batch size of 8.'''
    
    model.save("model.tflearn")  # acc: 0.5106 -- iter: 401/401 for 1000 epochs
                                 # acc: 0.4652 -- iter: 401/401 for 1500 epochs acc: 0.5106 -- iter: 401/401

Training Step: 50999  | total loss: 1.17452 | time: 2.467s
| Adam | epoch: 1000 | loss: 1.17452 - acc: 0.4474 -- iter: 400/401
Training Step: 51000  | total loss: 1.15262 | time: 2.480s
| Adam | epoch: 1000 | loss: 1.15262 - acc: 0.4526 -- iter: 401/401
--
INFO:tensorflow:C:\Users\Kranthi Kumar\chatBotMajorProject_YT\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


# Create function to give input to the Model.

In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

# Create GUI Chat window and other necessary components with tkinter

In [11]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

# Function to remove special chars and symbols

In [12]:
import re
def remove_special(m):
    m1=m.replace(","," ")
    nestr = re.sub(r'[^a-zA-Z0-9 ]',r'',m1)
    return nestr

# Function to get output from the model

In [13]:
def chatbot_response(msg):
    results = model.predict([bag_of_words(remove_special(msg), words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if(results[results_index]>0.7):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                t=tg['tag']
                responses = tg['responses']
                syms=tg["patterns"]
                info=","
                info=info.join(syms[1:])
        
        if t not in ["greetings","name","goodbye","thanks","options","diseases"]:
            return random.choice(responses),info
        else:
            return random.choice(responses),""
    else:
        return "Sorry,I'm unable to figure out.Please add some more symptoms.",""


In [14]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    s=msg
    s=s.replace(" ","")
    
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You: " + msg + '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    if s=="":
        ChatLog.insert(END, "DocBot: " + "You entered nothing.Please enter your symptoms." + '\n\n')
    
    elif s.isnumeric():
        ChatLog.insert(END, "DocBot: " + "You entered number.Please enter your symptoms." + '\n\n')
    
    elif s in ".,!@#$%^&*()_+={[}];:<>?":
        ChatLog.insert(END, "DocBot: " + "You entered a special symbol.Please enter your symptoms." + '\n\n')
 
    else:
        res,add_info=chatbot_response(msg)

        if res=="Sorry,I'm unable to figure out.Please add some more symptoms.":
            if s.isalnum():
                flag=0
                for i in symptoms:
                    if(msg.lower()==i.lower()):
                        ChatLog.insert(END, "DocBot: " + res + '\n\n')
                        flag=1
                        break
                if flag==0:
                    ChatLog.insert(END, "DocBot: " + "You entered alphanumeric.Try again with the symptoms I have known." + '\n\n')
            else:
                ChatLog.insert(END, "DocBot: " + "Sorry,I can't understand you.Try again entering your symptoms." + '\n\n')

                
        else:
            ChatLog.insert(END, "DocBot: " + res + '\n\n')
            if add_info!="":
                ChatLog.insert(END, "DocBot: It's symptoms include : " + add_info + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)
    
    if msg.lower()=="quit":
        base.destroy() 

In [15]:
base = Tk()
base.title("DocBot")
base.geometry("400x500")
base.resizable(width=TRUE, height=TRUE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

In [29]:
#LancasterStemmer
print(words)

['!', '(', ')', ',', 'a', 'abdom', 'abdomin', 'abnorm', 'about', 'acid', 'acn', 'acut', 'alcohol', 'allergy', 'alt', 'an', 'and', 'anxy', 'anyon', 'appetit', 'around', 'arthrit', 'asthm', 'attack', 'awesom', 'b', 'back', 'bal', 'be', 'behind', 'bel', 'benign', 'blackhead', 'blad', 'blee', 'blist', 'blood', 'bloody', 'blur', 'body', 'bowel', 'bp', 'brain', 'breath', 'breathless', 'brittl', 'bronch', 'bru', 'buddy', 'burn', 'bye', 'c', 'calf', 'can', 'cas', 'cerv', 'chat', 'chest', 'chick', 'chil', 'cholestas', 'chronic', 'cold', 'com', 'common', 'congest', 'constip', 'consum', 'cont', 'continu', 'cough', 'could', 'covid-19', 'cramp', 'crust', 'd', 'dark', 'dehydr', 'dengu', 'dent', 'depress', 'diabet', 'diarrhoe', 'difficul', 'dimorph', 'dischrom', 'discomfort', 'diseas', 'dist', 'distort', 'disturb', 'dizzy', 'do', 'drug', 'dry', 'dur', 'dust', 'e', 'enlarg', 'erupt', 'excess', 'excessive_hunger', 'extrem', 'ey', 'fac', 'fail', 'famy', 'fast', 'fath', 'fatigu', 'feel', 'fev', 'fil', 'f

In [30]:
#LancasterStemmer
len(words)

299

In [6]:
#WordNetLemmatizer
print(words)

['!', '(', ')', ',', 'a', 'abdomen', 'abdominal', 'abnormal', 'about', 'acidity', 'acne', 'acute', 'alcohol', 'alcoholic', 'allergy', 'altered', 'anal', 'and', 'anus', 'anxiety', 'anyone', 'appetite', 'around', 'arthritis', 'asthma', 'attack', 'awesome', 'b', 'back', 'balance', 'be', 'behind', 'belly', 'benign', 'blackhead', 'bladder', 'bleeding', 'blister', 'blood', 'bloody', 'blurred', 'body', 'bowel', 'bp', 'brain', 'breathing', 'breathlessness', 'brittle', 'bronchial', 'bruising', 'buddy', 'burning', 'bye', 'c', 'calf', 'can', 'case', 'cervical', 'chatting', 'chest', 'chicken', 'chill', 'cholestasis', 'chronic', 'cold', 'coma', 'common', 'concentration', 'congestion', 'constipation', 'consumption', 'continuous', 'cough', 'could', 'covid-19', 'cramp', 'crust', 'd', 'dark', 'dehydration', 'dengue', 'dent', 'depression', 'diabetes', 'diarrhoea', 'difficulty', 'dimorphic', 'dischromic', 'discomfort', 'disease', 'distention', 'distorted', 'disturbance', 'dizziness', 'do', 'drug', 'dryin

In [7]:
#WordNetLemmatizer
len(words)

318

In [16]:
import anvil.server
anvil.server.connect("NTBDZOFMUYGOPQV2N4B5ZIV2-XCKWEJIILBT3FLCU")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [17]:
@anvil.server.callable


def DocBot(msg):
    import nltk
    from nltk.stem import WordNetLemmatizer
    import re
    lemmatizer = WordNetLemmatizer()
    
    def remove_special(m):
        m1=m.replace(","," ")
        nestr = re.sub(r'[^a-zA-Z0-9 ]',r'',m1)
        return nestr
    
    def bag_of_words(s, words):
        bag = [0 for _ in range(len(words))]
        s_words = nltk.word_tokenize(s)
        s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]
        for se in s_words:
            for i, w in enumerate(words):
                if w == se:
                    bag[i] = 1
        return numpy.array(bag)
                
    results = model.predict([bag_of_words(remove_special(msg), words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if(results[results_index]>0.7):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                t=tg['tag']
                responses = tg['responses']
                syms=tg["patterns"]
                info=","
                info=info.join(syms[1:])
        if t not in ["greetings","name","goodbye","thanks","options","diseases"]:
            return random.choice(responses),info
        else:
            return random.choice(responses),""
    else:
        return "Sorry,I'm unable to figure out.Please add some more symptoms.",""

In [18]:
anvil.server.get_app_origin()

'https://docbot.anvil.app'